In [2]:
# Modules
# Run this
import tarfile
import zipfile as zf
import pandas as pd
import io
import networkx as nx
import matplotlib.pyplot as plt
import os
from PyPDF2 import PdfReader

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from pathlib import Path
import tempfile
import pprint
from science_parse_api.api import parse_pdf
from IPython.display import Image
from science_parse_api.test_helper import test_data_dir

import pickle

# dataset D1

In [3]:
# load dataset D1
# already run. Do not run again
df_d1 = pd.read_csv('paperid_title.csv')
print(df_d1)

        paperid                                              Title  \
0             1  Toward identifying inter-project clone sets fo...   
1             2  A proposal for chemical information retrieval ...   
2             3  What does psychology and psychiatry need from ...   
3             4  Scalable detection of missed cross-function re...   
4             5  Exploiting information context to improve prod...   
...         ...                                                ...   
148923   148924  Exploiting reference section to classify paper...   
148924   148925                                       Instant IDEs   
148925   148926  Advancing the discovery of unique column combi...   
148926   148927  A Monte Carlo algorithm for cold start recomme...   
148927   148928                                              BLOCK   

                                                 Subtitle  
0                                                     NaN  
1                                      

In [5]:
# already run. Do not run again
df_d1.describe()

,paperid
count,148928.000000
mean,74464.500000
std,42991.954782
min,1.000000
25%,37232.750000
50%,74464.500000
75%,111696.250000
max,148928.000000


In [6]:
# already run. Do not run again
df_d1.head()

,paperid,Title,Subtitle
0,1,Toward identifying inter-project clone sets fo...,NaN
1,2,A proposal for chemical information retrieval ...,NaN
2,3,What does psychology and psychiatry need from ...,an end-user perspective
3,4,Scalable detection of missed cross-function re...,NaN
4,5,Exploiting information context to improve prod...,NaN


In [7]:
# fillna() helps to replace all NaN values in a DataFrame
# already run. Do not run again
df_d1['Subtitle'].fillna("", inplace=True)
df_d1.head()

,paperid,Title,Subtitle
0,1,Toward identifying inter-project clone sets fo...,
1,2,A proposal for chemical information retrieval ...,
2,3,What does psychology and psychiatry need from ...,an end-user perspective
3,4,Scalable detection of missed cross-function re...,
4,5,Exploiting information context to improve prod...,


In [8]:
# Using + operator to combine two columns
# already run. Do not run again
df_d1["Title"] = df_d1['Title'].astype(str) +" "+ df_d1["Subtitle"].astype(str)
df_d1.head()

,paperid,Title,Subtitle
0,1,Toward identifying inter-project clone sets fo...,
1,2,A proposal for chemical information retrieval ...,
2,3,What does psychology and psychiatry need from ...,an end-user perspective
3,4,Scalable detection of missed cross-function re...,
4,5,Exploiting information context to improve prod...,


In [9]:
# already run. Do not run again
print(df_d1.iloc[2]["Title"])

What does psychology and psychiatry need from mobile systems an end-user perspective


In [10]:
# drop a column
# already run. Do not run again
df_d1 = df_d1.drop(columns = ['Subtitle'])
df_d1.head()

,paperid,Title
0,1,Toward identifying inter-project clone sets fo...
1,2,A proposal for chemical information retrieval ...
2,3,What does psychology and psychiatry need from ...
3,4,Scalable detection of missed cross-function re...
4,5,Exploiting information context to improve prod...


In [11]:
df_d1.duplicated(subset = None, keep = 'first')

0         False
1         False
2         False
3         False
4         False
          ...  
148923    False
148924    False
148925    False
148926    False
148927    False
Length: 148928, dtype: bool

In [13]:
# convert pandas df into iterative data
D1 = df_d1.to_dict('list')

""" D1 is of structure: 
{
"paperid" : [ids...],
"Title" : [titles...]
}
"""

' D1 is of structure: \n{\n"paperid" : [ids...],\n"Title" : [titles...]\n}\n'

# dataset 2

In [ ]:
# load zipped tarfile
# already run. Do not run again
tf_d2 = tarfile.open("sampled_data_for_bda.tar.gz", "r:gz")
print(tf_d2)

In [ ]:
# extract tarfile
# already run. Do not run again
tf_d2.extractall()

In [14]:
# generate a list of pdf files in D2
# already run. Do not run again
pdf_list = os.listdir(r"C:\Users\lenovo\Downloads\Big Data Analytics\Assignment 1\sampled_data_for_bda")
print(pdf_list)

['0704.0213.pdf', '0704.2902.pdf', '0705.0564.pdf', '0705.1309.pdf', '0705.1759.pdf', '0705.1922.pdf', '0705.2145.pdf', '0706.0300.pdf', '0706.1755.pdf', '0706.2725.pdf', '0706.4044.pdf', '0707.2998.pdf', '0707.3666.pdf', '0707.4166.pdf', '0707.4258.pdf', '0708.0741.pdf', '0708.1579.pdf', '0708.3811.pdf', '0709.0145.pdf', '0709.0674.pdf', '0709.1023.pdf', '0709.3094.pdf', '0710.1280.pdf', '0710.3901.pdf', '0710.4652.pdf', '0710.4670.pdf', '0710.4724.pdf', '0710.4815.pdf', '0710.4840.pdf', '0711.0692.pdf', '0711.1269.pdf', '0711.1533.pdf', '0711.1786.pdf', '0711.2546.pdf', '0711.2895.pdf', '0711.2897.pdf', '0712.0411.pdf', '0712.0554.pdf', '0712.1310.pdf', '0712.1521.pdf', '0712.1863.pdf', '0712.2592.pdf', '0712.2872.pdf', '0712.3203.pdf', '0801.0514.pdf', '0801.1416.pdf', '0801.2405.pdf', '0802.1369.pdf', '0802.2823.pdf', '0802.2834.pdf', '0802.2845.pdf', '0802.3074.pdf', '0802.3092.pdf', '0803.0450.pdf', '0803.0528.pdf', '0803.1985.pdf', '0803.2212.pdf', '0803.2615.pdf', '0803.3422.pd

# Read pdf files

In [19]:
pdf_obj_list = []

In [17]:
def read_pdf(pdf_name, host = 'http://127.0.0.1', port = '8080'):
    """ input: string (pdfname.pdf)
        output: dict
                {'title': string,
                'references': [strings]}
        run command for docker: 
docker run -p 127.0.0.1:8080:8080 --rm --init ucrel/ucrel-science-parse:3.0.1
    """
    
    path = r"C:\Users\lenovo\Downloads\Big Data Analytics\Assignment 1\sampled_data_for_bda\{fname}".format(fname = pdf_name)
    
    output_dict = parse_pdf(host, Path(path).resolve(), port = port)
    
    pp = pprint.PrettyPrinter(indent = 4)
    
    flag = True
    
    try:
        metadata = {}

        title = output_dict['title']
        metadata['title'] = title

        references = output_dict['references']
        metadata['references'] = []

        for reference in references:
            metadata['references'].append(reference['title'])
            
    except:
        flag = False

    return (flag, metadata)

In [18]:
# example. Do not run
read_pdf('0705.1309.pdf')

URL: http://127.0.0.1:8080/v1. 0705.1309.pdf failed due to the following error:
Traceback (most recent call last):
  File "C:\Users\lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\lenovo\anaconda3\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\lenovo\anaconda3\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request

(False, {})

In [20]:
def batch_read(index):
    """ input: int (index in pdf_list)
        output: none (appends metadata in pdf_obj_list)
    """
    for i in range(index, index+1000):
        print(pdf_list[i])
        flag, metadata = read_pdf(pdf_list[i])
        if flag:
            pdf_obj_list.append(metadata)

In [ ]:
# already run. Do not run again
batch_read(0)

In [ ]:
# already run. Do not run again
batch_read(1000)

In [ ]:
# already run. Do not run again
batch_read(2000)

In [ ]:
# already run. Do not run again
batch_read(3000)

In [ ]:
# already run. Do not run again
batch_read(4000)

In [ ]:
# already run. Do not run again
batch_read(5000)

In [ ]:
# already run. Do not run again
batch_read(6000)

In [ ]:
# already run. Do not run again
batch_read(7000)

In [ ]:
# already run. Do not run again
batch_read(8000)

In [ ]:
# already run. Do not run again
batch_read(9000)

# Class and its methods for data dumps

In [21]:
class MyClass():
    def __init__(self, param):
        self.param = param

In [22]:
def save_object(obj):
    try:
        with open("Part1_D2.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

In [23]:
def save_mp(obj):
    try:
        with open("Part1_mp.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

In [55]:
def save_mp_part2(obj):
    try:
        with open("Part1_mp_part2.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

In [65]:
def save_citation_dict(obj):
    try:
        with open("Part1_citation_dict.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

In [24]:
obj = MyClass(pdf_obj_list)
save_object(obj)

In [25]:
def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)

In [26]:
D2 = load_object("Part1_D2.pickle").param

In [27]:
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(D2[0])

{   'references': [   'Tensor product multiplicities and convex polytopes in '
                      'partition space',
                      'On the computation of Clebsch-Gordan coefficients and '
                      'the dilation effect',
                      'Polynomial algorithms for computing the Smith and '
                      'Hermite normal forms of an integer matrix',
                      'A new polynomial-time algorithm for linear programming',
                      'A polynomial algorithm for linear programming',
                      'Stretched Littlewood-Richardson coefficients and Kostka '
                      'coefficients',
                      'The Honeycomb model of GLn(C) tensor products I: proof '
                      'of the saturation conjecture',
                      'Honeycombs and sums of Hermitian matrices',
                      'Geometric complexity theory III, on deciding positivity '
                      'of Littlewood-Richardson coefficients, 

# Approximate string matching algo


In [28]:
match_list = []

In [29]:
for file in D2:
    print("Title D2: ",file["title"])
    val = (process.extractOne(file["title"], D1["Title"]))
    if val[-1] > 90:
        file_in_D1 = {}
        file_in_D1["Title"] = val[0]
        file_in_D1["paperid"] = D1["paperid"][D1["Title"].index(val[0])]
        file_in_D1["References"] = file["references"]
        match_list.append(file_in_D1)
    print(len(match_list))
print(match_list)

Title D2:  Geometric Complexity Theory V: On deciding nonvanishing of a generalized Littlewood-Richardson coefficient
0
Title D2:  Recommending Related Papers Based on Digital Library Access Records
0
Title D2:  Rate Bounds for MIMO Relay Channels Using Message Splitting
0
Title D2:  Crystallization in Large Wireless Networks
0
Title D2:  Elementary transformation analysis for Array-OL
0
Title D2:  For a long time V/Q scanning has been the imaging protocol of choice for the evaluation of patients with suspected PE. The procedure has two parts: Perfusion and ventilation which enables visualization the blood and air flow in the lungs. 1) Perfusion: The patient is injected with 5mCi Technetium-99m MAA (macroaggregated albumin). This allows the radiologist to visualise the flow of blood in the patients lungs. 2) Ventilation: The patient is asked to inhale 20mCi of Technetium-99m DTPA (diethylenetriamine-pentaacetic acid) labelled aerosol which indicates airflow
0
Title D2:  Implementing En

2
Title D2:  Minimizing Maximum Response Time and Delay Factor in Broadcast Scheduling
2
Title D2:  Some Considerations on Universality
2
Title D2:  Quantifying the Availability of TV White Spaces for Cognitive Radio Operation in the UK
2
Title D2:  Geometrical Interpretation of the Master Theorem for Divide-and-conquer Recurrences
2
Title D2:  Integrating Genetic Algorithm, Tabu Search Approach for Job Shop Scheduling
2
Title D2:  An Improved Algorithm based on Shannon-Happ Formula for Calculating Transfer Function from Signal Flow Graph and Its Visualization
2
Title D2:  Capacity Regions and Sum-Rate Capacities of Vector Gaussian Interference Channels
2
Title D2:  Generalized Collective Inference with Symmetric Clique Potentials
2
Title D2:  A Process Planning System with Feature Based Neural Network Search Strategy for Aluminum Extrusion Die Manufacturing
2
Title D2:  Deterministic Polynomial Time Algorithms for Matrix Completion Problems
2
Title D2:  Diffusion of scientific credits

2
Title D2:  A Unified Formal Description of Arithmetic and Set Theoretical Data Types
2
Title D2:  Flows in One-Crossing-Minor-Free Graphs
2
Title D2:  Scheduling for Optimal Rate Allocation in Ad Hoc Networks Subject to Heterogeneous Delay Constraints
2
Title D2:  On the Applicability of Post’s Lattice∗
2
Title D2:  Relative entropy via non-sequential recursive pair substitution
2
Title D2:  Introduction to Xgrid: Cluster Computing for Everyone
2
Title D2:  Security Aware Mobile Web Service Provisioning
2
Title D2:  Fast Moment Estimation in Data Streams in Optimal Space
3
Title D2:  Competitive Analysis of Minimum-Cut Maximum Flow Algorithms in Vision Problems
3
Title D2:  Gbits/s physical-layer stream ciphers based on chaotic light
3
Title D2:  Approximation Algorithms for Secondary Spectrum Auctions∗
4
Title D2:  Synchronization and Control in Intrinsic and Designed Computation: An Information-Theoretic Analysis of Competing Models of Stochastic Computation
4
Title D2:  The Coarse

7
Title D2:  Bounds on the Capacity of the Relay Channel with Noncausal State at Source
7
Title D2:  Evolution of Computer Virus Concealment and Anti-Virus Techniques: A Short Survey
7
Title D2:  Amortised Resource Analysis with Separation Logic
7
Title D2:  Load-Balancing Spatially Located Computations using Rectangular Partitions∗
7
Title D2:  On the exponential decay of the Euler-Bernoulli beam with boundary energy dissipation
7
Title D2:  Optimal Threshold–Based Multi–Trial Error/Erasure Decoding with the Guruswami–Sudan Algorithm
7
Title D2:  On Optimal Binary One-Error-Correcting Codes of Lengths2m
7
Title D2:  Improving digital signal interpolation: L2-optimal kernels with kernel-invariant interpolation speed
7
Title D2:  A Short Course on Frame Theory
7
Title D2:  A MACHINE LEARNING BASED ANALYTICAL FRAMEWORK FOR SEMANTIC ANNOTATION REQUIREMENTS
7
Title D2:  A Topology Independent and Closed Form Maximum Likelihood Estimator to End-to-End Loss Rate Estimation
7
Title D2:  Unive

9
Title D2:  Social Capital and Individual Performance: A Study of Academic Collaboration
9
Title D2:  Isomorphisms of types in the presence of higher-order references
9
Title D2:  Chapter # KM AND WEB 2.0 METHODS FOR PROJECT-BASED LEARNING. MESHAT: A MONITORING AND EXPERIENCE SHARING TOOL
9
Title D2:  Certifying Graph Expansion and Non-Uniform Sparsity via Generalized Spectra
9
Title D2:  Online Learning for Classification of Low-rank Representation Features and Its Applications in Audio Segment Classification
9
Title D2:  Bounded-Distance Network Creation Games⋆
9
Title D2:  Towards realistic modeling of IP-level routing topology dynamics
9
Title D2:  On the optimal frequency selectivity to maximize multiuser diversity in an OFDMA scheduling system
9
Title D2:  On Multiple Round Sponsored Search Auctions with Budgets
9
Title D2:  Design and Analysis of Coalitions in Data Swarming Systems
9
Title D2:  CONTINUITY IN INFORMATION ALGEBRAS: A SURVEY ON THE RELATIONSHIP BETWEEN TWO TYPES O

...

81
Title D2:  Alternating Direction Method of Multipliers for Quantization
81
Title D2:  Space efficient merging of de Bruijn graphs and Wheeler graphs∗
81
Title D2:  An Efficient Approach for Reviewing Security-Related Aspects in Agile Requirements Specifications of Web Applications
81
Title D2:  Simple is Better! Lightweight Data Augmentation for Low Resource Slot Filling and Intent Classification
81
Title D2:  Intraoperative Liver Surface Completion with Graph Convolutional VAE
81
Title D2:  Silicon Dating
81
Title D2:  REAL-TIME PLANT HEALTH ASSESSMENT VIA IMPLEMENTING CLOUD-BASED SCALABLE TRANSFER LEARNING ON AWS DEEPLENS
81
Title D2:  A Conservative High-Order Method Utilizing Dynamic Transfinite Mortar Elements for Flow Simulation on Curved Sliding Meshes
81
Title D2:  Physics-Informed Neural Networks for Nonhomogeneous Material Identification in Elasticity Imaging
81
Title D2:  Virtual Image Correlation uncertainty
81
Title D2:  Multi-instance Domain Adaptation for Vaccine Adve

82
Title D2:  A Study for Universal Adversarial Attacks on Texture Recognition
82
Title D2:  ON NONNEGATIVE MATRIX AND TENSOR DECOMPOSITIONS FOR COVID-19 TWITTER DYNAMICS
82
Title D2:  Adversarial Attack and Defense of Structured Prediction Models
82
Title D2:  A Large-Scale Mixed-Methods Analysis of Live Streaming Based Remote Education Experience in Chinese Colleges during the COVID-19 Pandemic
82
Title D2:  Corpora Evaluation and System Bias Detection in Multi-document Summarization
82
Title D2:  Pruning Redundant Mappings in Transformer Models via Spectral-Normalized Identity Prior
82
Title D2:  JSSS: free Japanese speech corpus for summarization and simplification
82
Title D2:  Modulated Fusion using Transformer for Linguistic-Acoustic Emotion Recognition
82
Title D2:  Learning to Generalize for Sequential Decision Making
82
Title D2:  Finding Ethereum Smart Contracts Security Issues  by Comparing History Versions
82
Title D2:  DaNetQA: a yes/no Question Answering Dataset for the 

82
Title D2:  Improving the Iterative Closest Point Algorithm using Lie Algebra
82
Title D2:  Riemannian Langevin Algorithm for Solving Semidefinite Programs
82
Title D2:  On Compressed Sensing Matrices Breaking the Square-Root Bottleneck
82
Title D2:  MICRONETS: NEURAL NETWORK ARCHITECTURES
82
Title D2:  Linking Entities to Unseen Knowledge Bases with Arbitrary Schemas
82
Title D2:  Distilling Dense Representations for Ranking using Tightly-Coupled Teachers
82
Title D2:  GAN based Unsupervised Segmentation: Should We Match the Exact Number of Objects
82
Title D2:  Lung Nodule Classification Using Biomarkers, Volumetric Radiomics and 3D CNNs
82
Title D2:  The Polynomial Method is Universal for Distribution-Free Correlational SQ Learning
82
Title D2:  Towards falsifiable interpretability research
82
Title D2:  Comprehensive Attention Self-Distillation for Weakly-Supervised Object Detection
82
Title D2:  Codes over integers, and the singularity of random matrices with large entries
82
Ti

82
Title D2:  State-of-the-Art and Future Challenges
82
Title D2:  Learning Continuous System Dynamics from Irregularly-Sampled Partial Observations
82
Title D2:  The Hierarchical Chinese Postman Problem: the slightest disorder makes it hard,  yet disconnectedness is manageable
82
Title D2:  EGO-Swarm: A Fully Autonomous and Decentralized Quadrotor Swarm System in Cluttered Environments
82
Title D2:  Guided Source Separation
82
Title D2:  Speaker De-identification System using Autoencoders and Adversarial Training
82
Title D2:  The quasi-reversibility method to numerically solve an inverse source problem for hyperbolic equations
82
Title D2:  Resource Allocation in One-dimensional Distributed Service Networks with Applications
82
Title D2:  Expressiveness of Neural Networks Having Width Equal or Below the Input Dimension
82
Title D2:  Selectively Precoded Polar Codes
82
Title D2:  Fast Galerkin Method for Solving Helmholtz Boundary Integral Equations on Screens 1Fast Galerkin Method fo

KeyboardInterrupt: 

In [30]:
mp = MyClass(match_list)
save_mp(mp)

In [31]:
mp = load_object("Part1_mp.pickle").param

# Algorithm for citation dictionary

In [54]:
for file in mp:
    print()
    print("Title :", file['Title'])
    ref_list = []
    for ref in file['References']:
        print("Reference Title: ", ref)
        val = (process.extractOne(ref, D1["Title"]))
        print("Result: ", val)
        if val[-1] >= 90:
            paper_id = D1["paperid"][D1["Title"].index(val[0])]
            ref_list.append(paper_id)
    print("Updates :", len(ref_list))
    file['References'] = ref_list
    pp = pprint.PrettyPrinter(indent = 4)
    pp.pprint(file)


Title : Evolutionary Optimization in Code-Based Test Compression 
Reference Title:  Test vector decompression using cyclical scan chains and its application to testing core-based designs
Result:  ('Teaching push-down automata and turing machines ', 86)
Reference Title:  Scan vector compression/decompression using statistical coding
Result:  ('Towards artificial flavor perception of black tea An approach using fusion of electronic nose and electronic tongue responses with fuzzy ART classification technique', 86)
Reference Title:  Test data compression for system-on-a-chip using golomb codes
Result:  ("Evaluation of Thailand's Regional Hospital Efficiency an Application of Data Envelopment Analysis", 86)
Reference Title:  Frequency-directed run length codes (FDR) codes with application to system-on-achip test data compression
Result:  ('Effect of word density on measuring words association ', 86)
Reference Title:  Improving compression ratio, area overhead, and test application time for

Result:  ('Faster relaxed multiplication ', 63)
Reference Title:  New Series
Result:  ('MOSES exploring new ground in media and post-conflict reconciliation', 86)
Reference Title:  A divisor problem
Result:  ('A proposal for chemical information retrieval evaluation ', 86)
Reference Title:  The complexity of a scheme of functional elements simulating the multiplication of integers
Result:  ('A proposal for chemical information retrieval evaluation ', 86)
Updates : 0
{   'References': [],
    'Title': 'Fast integer multiplication using modular arithmetic ',
    'paperid': 31571}

Title : Fast moment estimation in data streams in optimal space 
Reference Title:  On the surprising behavior of distance metrics in high dimensional spaces
Result:  ('CS262 a breadth-second survey of informatic CS', 86)
Reference Title:  Tracking join and self-join sizes in limited storage
Result:  ('What does psychology and psychiatry need from mobile systems an end-user perspective', 86)
Reference Title:  Th

Result:  ('Scalable detection of missed cross-function refactorings ', 86)
Reference Title:  Strong edge coloring for channel assignment in wireless radio networks
Result:  ('Certificates for mobile code security ', 86)
Reference Title:  Dimsumnet: New directions in wireless networking using coordinated dynamic spectrum access
Result:  ('Display power management policies in practice ', 86)
Reference Title:  Truthful mechanisms for exhibitions
Result:  ('Toward identifying inter-project clone sets for building useful libraries ', 86)
Reference Title:  Truthful randomized mechanisms for combinatorial auctions
Result:  ('Truthful randomized mechanisms for combinatorial auctions ', 100)
Reference Title:  Online capacity maximization in wireless networks
Result:  ('Online capacity maximization in wireless networks ', 100)
Reference Title:  Oblivious interference scheduling
Result:  ('Oblivious interference scheduling ', 100)
Reference Title:  Improved algorithms for latency minimization in 

Result:  ('What does psychology and psychiatry need from mobile systems an end-user perspective', 86)
Reference Title:  Incentive compatible ranking systems
Result:  ('Incentive compatible ranking systems ', 100)
Reference Title:  Efficient influence maximization in social networks
Result:  ('Efficient influence maximization in social networks ', 100)
Reference Title:  Multi-part pricing of public goods
Result:  ('Scalable detection of missed cross-function refactorings ', 86)
Reference Title:  Quality concious and truthful query incentive networks
Result:  ('What does psychology and psychiatry need from mobile systems an end-user perspective', 86)
Reference Title:  Mining the network value of customers
Result:  ('Mining the network value of customers ', 100)
Reference Title:  Learning influence probabilities in social networks
Result:  ('Learning influence probabilities in social networks ', 100)
Reference Title:  Maximizing spread of influence through a social network
Result:  ('Maxi

Result:  ('Social Network, Web Forum, or Task Market? Comparing Different Crowd Genres for Design Feedback Exchange', 86)
Updates : 1
{   'References': [45344],
    'Title': 'Agile and pro-active public administration as a collaborative '
             'networked organization ',
    'paperid': 2031}

Title : On the power of (even a little) centralization in distributed processing 
Reference Title:  State space collapse with application to heavy traffic limits for multiclass queueing networks
Result:  ('A proposal for chemical information retrieval evaluation ', 86)
Reference Title:  Markov Processes: Characterization and Convergence (2nd edition)
Result:  ('Feature interaction analysis in mobile phones on the borderline between application functionalities and platform components', 86)
Reference Title:  Queueing system with selection of the shortest of two queues: An asymptotic approach
Result:  ('Scalable detection of missed cross-function refactorings ', 86)
Reference Title:  The power

...

Result:  ('Informing online and mobile map design with the collective wisdom of cartographers ', 100)
Reference Title:  Mapping the World
Result:  ("Mapping the world's photos ", 90)
Reference Title:  Indooroutdoor image classification
Result:  ('Heart sounds classification using motif based segmentation ', 86)
Reference Title:  EnsembleMatrix: interactive visualization to support machine learning with multiple classifiers
Result:  ('Real-time out-of-core visualization of particle traces ', 86)
Reference Title:  No free lunch theorems for optimization
Result:  ('Toward identifying inter-project clone sets for building useful libraries ', 86)
Reference Title:  Location-Based Services. Technical Report
Result:  ('An energy balanced mobile agents based data dissemination protocol for wireless sensor networks ', 86)
Updates : 4
{   'References': [58935, 112208, 109673, 7795],
    'Title': 'Helping Computers Understand Geographically-Bound Activity '
             'Restrictions ',
    'paper

Result:  ('Ad Serving with Multiple KPIs ', 86)
Reference Title:  Obfuscated gradients give a false sense of security: Circumventing defenses to adversarial examples
Result:  ('A proposal for chemical information retrieval evaluation ', 86)
Reference Title:  Practical black-box attacks against machine learning
Result:  ('Towards artificial flavor perception of black tea An approach using fusion of electronic nose and electronic tongue responses with fuzzy ART classification technique', 86)
Reference Title:  Towards deep learning models resistant to adversarial attacks
Result:  ('Deep learning ', 90)
Reference Title:  Zoo: Zeroth order optimization based black-box attacks to deep neural networks without training substitute models
Result:  ('ZOO Zeroth Order Optimization Based Black-box Attacks to Deep Neural Networks without Training Substitute Models', 100)
Reference Title:  Adversarial classification
Result:  ('Adversarial classification ', 100)
Reference Title:  Boosting adversarial 

Result:  ('CS262 a breadth-second survey of informatic CS', 86)
Reference Title:  Distilling the knowledge in a neural network
Result:  ('Exploiting information context to improve productivity by modeling the domain specific tasks of the users ', 86)
Reference Title:  Quantum classifier with tailored quantum kernel
Result:  ('Peak power modeling for data center servers with switched-mode power supplies ', 86)
Reference Title:  An artificial neuron implemented on an actual quantum processor
Result:  ("Evaluation of Thailand's Regional Hospital Efficiency an Application of Data Envelopment Analysis", 86)
Reference Title:  Quantum Computing in the NISQ era and beyond
Result:  ('Quantum computing ', 90)
Reference Title:  Quantum Recommendation Systems
Result:  ('What does psychology and psychiatry need from mobile systems an end-user perspective', 86)
Reference Title:  Architectures for a quantum random access memory
Result:  ('Toward identifying inter-project clone sets for building usefu

Result:  ('Feature interaction analysis in mobile phones on the borderline between application functionalities and platform components', 86)
Reference Title:  Vulnerability of machine learning phases of matter
Result:  ('Exploiting information context to improve productivity by modeling the domain specific tasks of the users ', 86)
Reference Title:  Machine learning z2 quantum spin liquids with quasiparticle statistics
Result:  ('Ad Serving with Multiple KPIs ', 86)
Reference Title:  Realization of 2d spinorbit interaction and exotic topological orders in cold atoms
Result:  ('Effect of word density on measuring words association ', 86)
Reference Title:  Seeing topological order in time-offlight measurements
Result:  ('What we have here is a failure of companionship communication in goal-oriented team-mate games', 86)
Reference Title:  Direct probe of topological order for cold atoms
Result:  ('Toward identifying inter-project clone sets for building useful libraries ', 86)
Reference T

Result:  ('Scalable detection of missed cross-function refactorings ', 86)
Reference Title:  Sensitivity studies of developing convection in a cloud-resolving model
Result:  ('CS262 a breadth-second survey of informatic CS', 86)
Reference Title:  Sensitivity studies using a cloud-resolving model simulation of the tropical west pacific
Result:  ('A proposal for chemical information retrieval evaluation ', 86)
Reference Title:  Conservation properties of convection difference schemes
Result:  ('Exploiting information context to improve productivity by modeling the domain specific tasks of the users ', 86)
Reference Title:  Controls on precipitation and cloudiness in simulations of trade-wind cumulus as observed during rico
Result:  ('Scalable detection of missed cross-function refactorings ', 86)
Updates : 0
{   'References': [],
    'Title': 'A directive based hybrid met office NERC cloud model ',
    'paperid': 28425}


In [57]:
part2 = MyClass(mp)
save_mp_part2(part2)

In [59]:
part2 = load_object("Part1_mp_part2.pickle").param

In [70]:
citation_dict = {}

In [71]:
for file in part2:
    citation_dict[file['paperid']] = file['References']

In [73]:
citation_dict_obj = MyClass(citation_dict)
save_citation_dict(citation_dict_obj)
citation_dict_val = load_object("Part1_citation_dict.pickle").param
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(citation_dict_val)

{   2031: [45344],
    4025: [130133, 59980, 104017],
    5813: [   46492,
              9952,
              2250,
              91189,
              7278,
              69920,
              41821,
              30105,
              134687,
              104465,
              104465,
              76500,
              119938,
              47613,
              61782,
              5813],
    8228: [732, 30691],
    8690: [   90365,
              16173,
              101508,
              63858,
              2425,
              100153,
              139833,
              114021,
              124427,
              43428,
              19569],
    10778: [   56335,
               3810,
               21520,
               12821,
               1005,
               112544,
               12821,
               219,
               56335,
               1055,
               111242,
               519,
               9413,
               8351,
               7948,
               3810,
      